In [37]:
import os
import openai
import pandas as pd
from tqdm.auto import tqdm

In [45]:
# Ruta de la carpeta corpus que contiene todos los archivos CSV de las 640 llamadas
ruta_corpus = "ejercicio/corpus"

In [39]:
# Configuración de OpenAI api_key
api_key = "tu_api_key"  # Reemplaza "tu_api_key" con tu propia API key
openai.api_key = api_key

In [46]:
# CARGA Y LIMPIEZA DE CONVERSACIONES

# Crea lista para almacenar el contenido de cada archivo CSV
dataframes_consolidados = []

# Itera sobre todos los archivos CSV en la carpeta del corpus
for i, archivo_csv in enumerate(os.listdir(ruta_corpus)):
    if archivo_csv.endswith('.csv'):
        ruta_completa = os.path.join(ruta_corpus, archivo_csv)

        # Lee el contenido del archivo CSV como texto
        with open(ruta_completa, 'r', encoding='utf-8') as file:
            contenido_csv = file.read()

        # Elimina "Pregunta\n" de cada fila
        contenido_csv_sin_pregunta = contenido_csv.replace('Pregunta\n', '')

        # Agrega el contenido del CSV al DataFrame consolidado junto con el número de la llamada
        dataframes_consolidados.append({'Numero_Llamada': i, 'Conversacion': contenido_csv_sin_pregunta})

# Crea un DataFrame consolidado
df_consolidado = pd.DataFrame(dataframes_consolidados)

# Visualiza el DataFrame consolidado
print("Contenido del DataFrame consolidado")
print(df_consolidado)

Contenido del DataFrame consolidado
     Numero_Llamada                                       Conversacion
0                 0   Hola , mi nombre es Valeria . Disculpe , no e...
1                 1   Buenas tardes . punto Mi nombre es José Gutié...
2                 2   Hola . Buenas tardes . Habla con María Azcára...
3                 3   Buenas tardes , Francisco Moreno . , que mire...
4                 4   al final . Hola , buenas tardes con Karen . M...
..              ...                                                ...
635             635   barba . Buenas tardes . . tengo una consulta ...
636             636   Hola . Buenos días . Mi nombre es Paola . Le ...
637             637   Buenas tardes . Bienvenido . . mi nombre es E...
638             638   . Mira , tengo una pura . Es una compra onlin...
639             639   Buenas tardes . Bienvenida . Mi nombre es Edu...

[640 rows x 2 columns]


In [47]:
# PROCESAMIENTO DE CONVERSACIONES

# Crea lista para almacenar el tópico predominante de cada conversación
resultados = []

# Define tamaño del lote de conversaciones a procesar y hasta qué conversación procesar para no saturar el servidor de la API de OpenAI (modificables)
batch_size = 5
numero_llamada = 10

# Imprime mensaje antes de iniciar bucle
print(f"PROCESANDO CONVERSACIONES DE A LOTES DE A {batch_size} HASTA LA CONVERSACIÓN NÚMERO {numero_llamada}\n")

# Itera sobre los lotes de conversaciones
for i in tqdm(range(0, numero_llamada, batch_size)):
# Para correr todas las conversaciones, reemplazamos por:
# for i in tqdm(range(0, len(df_consolidado), batch_size)):

    # Selecciona un lote de conversaciones para procesar
    batch = df_consolidado.iloc[i:i+batch_size]

    # Crea una lista para almacenar los prompts de cada conversación en el lote
    prompts = []

    # Itera sobre las conversaciones en el lote y crea un prompt para cada una
    for index, (numero_llamada, conversacion) in enumerate(zip(batch['Numero_Llamada'], batch['Conversacion'])):

        # Construye el prompt con el número de llamada y su conversación, y lo agrega a la lista
        prompt = f"Interpretá la llamada {numero_llamada} y proporcioná su tópico, sin repetir la palabra tópico:\nConversación:\n{conversacion}\nRespuesta:"
        prompts.append(prompt)

        # Actualiza el índice del número de llamada dentro del lote
        numero_llamada = index + i

    # Crea lista para almacenar las respuestas de ese lote
    responses = []

    # Genera respuestas para los prompts usando OpenAI API
    for prompt in prompts:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=[
                {"role": "system", "content": "Sos un modelo de IA que determina los tópicos predominantes en cada conversación entre un cliente y un agente de contact center"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,  # Máximo de tokens generados
            temperature=0.1,  # Temperatura baja para limitar creatividad
            stop="\n",  # Frena la generación de texto cuando hay un salto de línea
            n=1,  # Genera una respuesta por cada conversación en el lote
        )

        # Agrega cada respuesta a la lista de resultados
        responses.append(response)


    # Itera sobre los resultados generados para cada conversación en el lote procesado
    for idx, (numero_llamada, response) in enumerate(zip(batch['Numero_Llamada'], responses)):

        # Imprime el número de llamada y el tópico de la conversación correspondiente a medida que se generan
        print(f"Llamada: {numero_llamada}, Tópico conversación: {response.choices[0].message['content'].strip()}")

        # Agrega el resultado a la lista de resultados para luego poder manipular esa data de acuerdo a lo que necesite el cliente
        resultados.append({
            "Numero de llamada": numero_llamada,
            "Tópico conversación": response.choices[0].message["content"].strip()
        })

# Imprime la lista con los resultados de todos esos lotes 
# La lista facilita la posterior manipulación de los resultados, de acuerdo a lo que el cliente requiera
print("\nRESULTADOS LOTES\n",resultados)


PROCESANDO CONVERSACIONES DE A LOTES DE A 5 HASTA LA CONVERSACIÓN NÚMERO 10



  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:06<00:06,  6.30s/it]

Llamada: 0, Tópico conversación: El tópico predominante en la conversación es el problema con la entrega de un pedido de alimentos, incluyendo la pérdida de la cadena de frío, la mala calidad de los productos recibidos y la necesidad de contactar al proveedor para resolver la situación.
Llamada: 1, Tópico conversación: La conversación parece estar relacionada con un pedido que no ha sido entregado a tiempo. El cliente expresa su preocupación por la falta de información sobre el estado de su pedido y la liberación del monto en su tarjeta. El agente tranquiliza al cliente asegurándole que el pedido será despachado en los próximos minutos. El tópico predominante de esta conversación es el seguimiento de un pedido.
Llamada: 2, Tópico conversación: Basándome en la conversación proporcionada, el tópico predominante es la consulta sobre el estado de un despacho o envío.
Llamada: 3, Tópico conversación: La conversación parece estar relacionada con un problema de entrega de pedidos, donde el cl

100%|██████████| 2/2 [00:13<00:00,  6.96s/it]

Llamada: 5, Tópico conversación: La conversación parece estar relacionada con un problema en un pedido de productos de alimentación, específicamente con la entrega de caldos bajos en sodio en lugar de sopas. La cliente expresa su confusión y la necesidad de resolver el error en el pedido. El agente ofrece su ayuda para encontrar una solución. El tópico predominante de esta conversación es la gestión de pedidos y la resolución de problemas relacionados con la entrega de productos.
Llamada: 6, Tópico conversación: La conversación parece estar relacionada con una compra de productos, en la que el cliente está reportando artículos faltantes en su pedido y también está consultando sobre la disponibilidad de otros productos. El agente parece estar buscando en el sistema la información sobre los productos faltantes y tratando de resolver la situación.
Llamada: 7, Tópico conversación: Basándome en la conversación proporcionada, el tópico predominante parece ser una consulta sobre una compra po